In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import heartpy as hp
from hrvanalysis import *
from pyEDA.main import *

from core.main import *

path = "eda_features.pkl"
print(os.path.isfile(path))
if os.path.isfile(path):
    return pd.read_pickle(path)

eda_channel = 36
sample_rate = 128
data = extract_data_by_video()
video_info = pd.read_csv("metadata_csv/video_list.csv")
# plt.figure(figsize=(12,4))
# plt.plot(x)
# plt.show()

ModuleNotFoundError: No module named 'pyEDA'

In [2]:
# EDA convex optimization novel feature extraction method
import ledapy

feature_names = ["conductance_error","conductance_min","conductance_max"]
peak_feature_names = ["onset","peaktime"]
results_frame = {c: [] for c in ['video_name', 'emotion', 'test_score'] + feature_names + peak_feature_names + ["mean_gsr"]}

eda_data = []
for video_id in range(40):
    print(video_id)
    # process EDA data
    section = data.loc[(data['video_id'] == video_id) & (data['channel_id'] == eda_channel)]
    X_input = np.array(section['channel_data'].tolist())

    for participant, x in enumerate(X_input):
        m, wd, eda_clean = process_statistical(x, use_scipy=False, sample_rate=128,
                                           new_sample_rate=20, segment_width=10,segment_overlap=0)
        # eda_data.append(m['mean_gsr'])
        try:
            phasicdata = ledapy.runner.getResult(x, 'phasicdata', sample_rate, downsample=4, optimisation=2)
        except Exception as e:
            print("participant "+participant+" for video "+video_id+" returns error")
            eda_data.append(np.array([0 for f in range(len(feature_names) + len(peak_feature_names) + 3)]))
            continue
        leda_data = ledapy.leda2.data
        leda_data2 = ledapy.leda2.trough2peakAnalysis
        trough_peak_data = [np.mean(getattr(leda_data2, tfeat)) for tfeat in peak_feature_names]
        core_data = [getattr(leda_data, feat_name) for feat_name in feature_names]

        new_row = np.array([participant, video_id] + core_data + trough_peak_data + [np.mean(m['mean_gsr'])])
        eda_data.append(new_row)

    row = video_info[video_info['Experiment_id'] == (video_id+1)].iloc[0]
    video_name = row['Artist'] + " - " + row['Title']

data_with_features_frame = pd.DataFrame(eda_data, columns=['participant', 'video']+
                                                          feature_names + peak_feature_names + ["mean_gsr"])
data_with_features_frame.to_pickle(path)

0
1


d:\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


2


d:\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


3
4
5
6


TypeError: can only concatenate str (not "int") to str

In [ ]:
# separate scoring from feature extraction
row = [video_name]
for emotion in ['Valence', 'Arousal', 'Dominance', 'Liking']:
    target_emotion = np.array(section[emotion].tolist())
    results_frame['video_name'].append(video_name)
    results_frame['emotion'].append(emotion)

    test_score, importance = run_test(eda_data, target_emotion, '')
    results_frame['test_score'].append(test_score)
    for i, f in enumerate(feature_names):
        results_frame[f].append(importance[i])
print(video_name)

results_frame = pd.DataFrame(results_frame)
print(results_frame)

